# nmap-tool
author: Paweł Popiołek <papopiolek@gmail.com>

## Table of contents
1. Introduction<br>
   1.1. [Components and libraries](#components)
2. JupytherNotebook version - step-by-step<br>
   2.1. [Scanning hosts](#scanning)<br>
   2.2. [Transfering scianning results to DataFrame](#wyniki_df)<br>
   2.3. [Listing vunerabilities for detected services](#wylistowanie)<br>
   2.4. [Saving the result to CSV file](#zapisanie)<br>
   2.5. [Sample query](#zapytanie)
3. Independent python script<br>
   3.1. [Description](#description)<br>
   3.2. [Script code](#code)


## 1. Introduction
Following tool:
- does nmap scanning for all given hosts
- veryfies all opened ports
- return versions of services found on opened ports
- lists vunerabilities for those services
- saves the result to CSV file for further analysis
  
<a id='components'></a>  
### 1.1. Components and libraries
- ```Nmap 7.92``` scanning tool
- ```Python 3.9.5``` script language
- ```python-nmap 0.7.1``` nmap library for python
- ```pandas 1.3.5``` data processing library for python
- ```nvdlib 0.5.6``` obtaining CVEs library for python


### Imports

In [2]:
import nmap
import pandas as pd
import nvdlib
from io import StringIO as sio
import numpy as np

## 2. JupytherNotebook version - step-by-step

Section 2 shows the thinking behind formation of the following script. It is recommended to use following steps as the independent solution. Main reason for that is that the results are very transparent in JupytherNotebook 
environment. However if you don't want to use JupytherNotebook Section 3 presents independent and fully functional python script.

<a id='scanning'></a>

### 2.1. Scanning hosts

In [3]:
f = open('list_of_hosts')
nm = nmap.PortScanner()
nm.scan(f.read())
print('nmap performed the following scan: ' + nm.command_line())


nmap performed the following scan: nmap -oX - -sV 192.168.1.26 192.168.1.37 192.168.1.38 192.168.1.39 192.168.1.104


<a id='wyniki_df'></a>

### 2.2. Transfering scianning results to DataFrame

In [695]:
data = sio(nm.csv())
df = pd.read_csv(data, sep=';')
df.head(12)

,host,hostname,hostname_type,protocol,port,name,state,product,extrainfo,reason,version,conf,cpe
0,192.168.1.104,pc-237.home,PTR,tcp,22,ssh,open,OpenSSH,protocol 1.99,syn-ack,2.9p2,10,cpe:/a:openbsd:openssh:2.9p2
1,192.168.1.104,pc-237.home,PTR,tcp,80,http,open,Apache httpd,(Unix) (Red-Hat/Linux) mod_ssl/2.8.4 OpenSSL/...,syn-ack,1.3.20,10,cpe:/a:apache:http_server:1.3.20
2,192.168.1.104,pc-237.home,PTR,tcp,111,rpcbind,open,NaN,RPC #100000,syn-ack,2,10,NaN
3,192.168.1.104,pc-237.home,PTR,tcp,139,netbios-ssn,open,Samba smbd,workgroup: MYGROUP,syn-ack,NaN,10,cpe:/a:samba:samba
4,192.168.1.104,pc-237.home,PTR,tcp,443,https,open,Apache/1.3.20 (Unix) (Red-Hat/Linux) mod_ssl/...,NaN,syn-ack,NaN,10,NaN
5,192.168.1.104,pc-237.home,PTR,tcp,1024,status,open,NaN,RPC #100024,syn-ack,1,10,NaN
6,192.168.1.26,macbook-air.home,PTR,tcp,5000,rtsp,open,AirTunes rtspd,NaN,syn-ack,610.19.1,10,cpe:/a:apple:airtunes:610.19.1
7,192.168.1.26,macbook-air.home,PTR,tcp,7000,rtsp,open,AirTunes rtspd,NaN,syn-ack,610.19.1,10,cpe:/a:apple:airtunes:610.19.1
8,192.168.1.26,macbook-air.home,PTR,tcp,49152,unknown,open,NaN,NaN,syn-ack,NaN,3,NaN
9,192.168.1.37,kali-linux-2021-1.home,PTR,tcp,4000,nomachine-nx,open,NoMachine NX Server remote desktop,NaN,syn-ack,7.7.4,10,cpe:/a:nomachine:nx_server:7.7.4


In [696]:
#filtering columns
df = df.loc[:, ['host', 'hostname', 'port', 'state', 'name', 'product', 'version', 'extrainfo', 'cpe']]
df.head(12)

,host,hostname,port,state,name,product,version,extrainfo,cpe
0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2
1,192.168.1.104,pc-237.home,80,open,http,Apache httpd,1.3.20,(Unix) (Red-Hat/Linux) mod_ssl/2.8.4 OpenSSL/...,cpe:/a:apache:http_server:1.3.20
2,192.168.1.104,pc-237.home,111,open,rpcbind,NaN,2,RPC #100000,NaN
3,192.168.1.104,pc-237.home,139,open,netbios-ssn,Samba smbd,NaN,workgroup: MYGROUP,cpe:/a:samba:samba
4,192.168.1.104,pc-237.home,443,open,https,Apache/1.3.20 (Unix) (Red-Hat/Linux) mod_ssl/...,NaN,NaN,NaN
5,192.168.1.104,pc-237.home,1024,open,status,NaN,1,RPC #100024,NaN
6,192.168.1.26,macbook-air.home,5000,open,rtsp,AirTunes rtspd,610.19.1,NaN,cpe:/a:apple:airtunes:610.19.1
7,192.168.1.26,macbook-air.home,7000,open,rtsp,AirTunes rtspd,610.19.1,NaN,cpe:/a:apple:airtunes:610.19.1
8,192.168.1.26,macbook-air.home,49152,open,unknown,NaN,NaN,NaN,NaN
9,192.168.1.37,kali-linux-2021-1.home,4000,open,nomachine-nx,NoMachine NX Server remote desktop,7.7.4,NaN,cpe:/a:nomachine:nx_server:7.7.4


<a id='wylistowanie'></a>

### 2.3. Listing vunerabilities for detected services

For each detected CPE the list of corresponding CVEs is going to be added. *Vunerabilities* column value will be *NaN* if a service version hadn't been detected or exact CPE had't been found. In case of successful CPE detecion *vunrabilities* column will contain a tuple: CVE.id and CVSS scoring data.

In [697]:
df['vunerabilities'] = df['cpe'].apply(lambda x: np.nan if x is np.nan else np.nan if x.count(':') == 3 else\
                        [(cve.id, cve.score) for cve in nvdlib.searchCVE(cpeName = x)])
df.head(12)

,host,hostname,port,state,name,product,version,extrainfo,cpe,vunerabilities
0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2,"[(CVE-2016-20012, [V3, 5.3, MEDIUM]), (CVE-202..."
1,192.168.1.104,pc-237.home,80,open,http,Apache httpd,1.3.20,(Unix) (Red-Hat/Linux) mod_ssl/2.8.4 OpenSSL/...,cpe:/a:apache:http_server:1.3.20,"[(CVE-2022-22721, [V3, 9.8, CRITICAL]), (CVE-2..."
2,192.168.1.104,pc-237.home,111,open,rpcbind,NaN,2,RPC #100000,NaN,NaN
3,192.168.1.104,pc-237.home,139,open,netbios-ssn,Samba smbd,NaN,workgroup: MYGROUP,cpe:/a:samba:samba,NaN
4,192.168.1.104,pc-237.home,443,open,https,Apache/1.3.20 (Unix) (Red-Hat/Linux) mod_ssl/...,NaN,NaN,NaN,NaN
5,192.168.1.104,pc-237.home,1024,open,status,NaN,1,RPC #100024,NaN,NaN
6,192.168.1.26,macbook-air.home,5000,open,rtsp,AirTunes rtspd,610.19.1,NaN,cpe:/a:apple:airtunes:610.19.1,[]
7,192.168.1.26,macbook-air.home,7000,open,rtsp,AirTunes rtspd,610.19.1,NaN,cpe:/a:apple:airtunes:610.19.1,[]
8,192.168.1.26,macbook-air.home,49152,open,unknown,NaN,NaN,NaN,NaN,NaN
9,192.168.1.37,kali-linux-2021-1.home,4000,open,nomachine-nx,NoMachine NX Server remote desktop,7.7.4,NaN,cpe:/a:nomachine:nx_server:7.7.4,[]


Getting service version in nmap-only approach isn't always possible, sometimes additional tools are needed (e.g. *smb_version* scanner for *samba*). For this reason column *status* is added -  *status* = *OK* means that service version had been successfully found and you can rely on presented data.

In [698]:
df['status'] = df['cpe'].apply(lambda x: 'Service not found' if x is np.nan else 'Version not found' if x.count(':') == 3 else\
                'OK' if x.count(':') == 4 else 'Unknown error')
df.head(12)

,host,hostname,port,state,name,product,version,extrainfo,cpe,vunerabilities,status
0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2,"[(CVE-2016-20012, [V3, 5.3, MEDIUM]), (CVE-202...",OK
1,192.168.1.104,pc-237.home,80,open,http,Apache httpd,1.3.20,(Unix) (Red-Hat/Linux) mod_ssl/2.8.4 OpenSSL/...,cpe:/a:apache:http_server:1.3.20,"[(CVE-2022-22721, [V3, 9.8, CRITICAL]), (CVE-2...",OK
2,192.168.1.104,pc-237.home,111,open,rpcbind,NaN,2,RPC #100000,NaN,NaN,Service not found
3,192.168.1.104,pc-237.home,139,open,netbios-ssn,Samba smbd,NaN,workgroup: MYGROUP,cpe:/a:samba:samba,NaN,Version not found
4,192.168.1.104,pc-237.home,443,open,https,Apache/1.3.20 (Unix) (Red-Hat/Linux) mod_ssl/...,NaN,NaN,NaN,NaN,Service not found
5,192.168.1.104,pc-237.home,1024,open,status,NaN,1,RPC #100024,NaN,NaN,Service not found
6,192.168.1.26,macbook-air.home,5000,open,rtsp,AirTunes rtspd,610.19.1,NaN,cpe:/a:apple:airtunes:610.19.1,[],OK
7,192.168.1.26,macbook-air.home,7000,open,rtsp,AirTunes rtspd,610.19.1,NaN,cpe:/a:apple:airtunes:610.19.1,[],OK
8,192.168.1.26,macbook-air.home,49152,open,unknown,NaN,NaN,NaN,NaN,NaN,Service not found
9,192.168.1.37,kali-linux-2021-1.home,4000,open,nomachine-nx,NoMachine NX Server remote desktop,7.7.4,NaN,cpe:/a:nomachine:nx_server:7.7.4,[],OK


In [699]:
#sorting CVEs by score
df['vunerabilities'] = df['vunerabilities'].apply(lambda x: np.nan if x is np.nan else sorted(x, key = lambda y: y[1][1], reverse = True))
df.head(12)

,host,hostname,port,state,name,product,version,extrainfo,cpe,vunerabilities,status
0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2,"[(CVE-2003-0693, [V2, 10.0, HIGH]), (CVE-2002-...",OK
1,192.168.1.104,pc-237.home,80,open,http,Apache httpd,1.3.20,(Unix) (Red-Hat/Linux) mod_ssl/2.8.4 OpenSSL/...,cpe:/a:apache:http_server:1.3.20,"[(CVE-2003-0789, [V2, 10.0, HIGH]), (CVE-2022-...",OK
2,192.168.1.104,pc-237.home,111,open,rpcbind,NaN,2,RPC #100000,NaN,NaN,Service not found
3,192.168.1.104,pc-237.home,139,open,netbios-ssn,Samba smbd,NaN,workgroup: MYGROUP,cpe:/a:samba:samba,NaN,Version not found
4,192.168.1.104,pc-237.home,443,open,https,Apache/1.3.20 (Unix) (Red-Hat/Linux) mod_ssl/...,NaN,NaN,NaN,NaN,Service not found
5,192.168.1.104,pc-237.home,1024,open,status,NaN,1,RPC #100024,NaN,NaN,Service not found
6,192.168.1.26,macbook-air.home,5000,open,rtsp,AirTunes rtspd,610.19.1,NaN,cpe:/a:apple:airtunes:610.19.1,[],OK
7,192.168.1.26,macbook-air.home,7000,open,rtsp,AirTunes rtspd,610.19.1,NaN,cpe:/a:apple:airtunes:610.19.1,[],OK
8,192.168.1.26,macbook-air.home,49152,open,unknown,NaN,NaN,NaN,NaN,NaN,Service not found
9,192.168.1.37,kali-linux-2021-1.home,4000,open,nomachine-nx,NoMachine NX Server remote desktop,7.7.4,NaN,cpe:/a:nomachine:nx_server:7.7.4,[],OK


In [700]:
#exploding records for improved readability
df_explode = df.explode('vunerabilities', ignore_index = False)
df_explode.reset_index(inplace=True)
df_explode.head(250)

,index,host,hostname,port,state,name,product,version,extrainfo,cpe,vunerabilities,status
0,0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2,"(CVE-2003-0693, [V2, 10.0, HIGH])",OK
1,0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2,"(CVE-2002-0640, [V2, 10.0, HIGH])",OK
2,0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2,"(CVE-2002-0639, [V2, 10.0, HIGH])",OK
3,0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2,"(CVE-2002-0083, [V2, 10.0, HIGH])",OK
4,0,192.168.1.104,pc-237.home,22,open,ssh,OpenSSH,2.9p2,protocol 1.99,cpe:/a:openbsd:openssh:2.9p2,"(CVE-2016-1908, [V3, 9.8, CRITICAL])",OK
...,...,...,...,...,...,...,...,...,...,...,...,...
230,20,192.168.1.39,ubuntu.home,80,open,http,lighttpd,1.4.28,NaN,cpe:/a:lighttpd:lighttpd:1.4.28,"(CVE-2013-4508, [V3, 7.5, HIGH])",OK
231,20,192.168.1.39,ubuntu.home,80,open,http,lighttpd,1.4.28,NaN,cpe:/a:lighttpd:lighttpd:1.4.28,"(CVE-2015-3200, [V3, 7.5, HIGH])",OK
232,20,192.168.1.39,ubuntu.home,80,open,http,lighttpd,1.4.28,NaN,cpe:/a:lighttpd:lighttpd:1.4.28,"(CVE-2011-4362, [V2, 5.0, MEDIUM])",OK
233,20,192.168.1.39,ubuntu.home,80,open,http,lighttpd,1.4.28,NaN,cpe:/a:lighttpd:lighttpd:1.4.28,"(CVE-2013-4560, [V2, 5.0, MEDIUM])",OK


<a id='zapisanie'></a>

### 2.4. Saving the result to CSV file

In [701]:
df_explode.to_csv('wynik_skanowania.csv')

<a id='zapytanie'></a>

### 2.5. Sample query
JupytherNotebook allows us to make simple queries for analysis.

In [702]:
#show only those columns:
df_show= df_explode.loc[:, ['host', 'port', 'state', 'name', 'product', 'version', 'extrainfo', 'status', 'vunerabilities']]
#conditions:
df_show.loc[(df_show['host'] == '192.168.1.39') & (df_show['status'] == 'OK')]

,host,port,state,name,product,version,extrainfo,status,vunerabilities
226,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2014-2323, [V3, 9.8, CRITICAL])"
227,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2019-11072, [V3, 9.8, CRITICAL])"
228,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2013-4559, [V2, 7.6, HIGH])"
229,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2018-19052, [V3, 7.5, HIGH])"
230,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2013-4508, [V3, 7.5, HIGH])"
231,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2015-3200, [V3, 7.5, HIGH])"
232,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2011-4362, [V2, 5.0, MEDIUM])"
233,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2013-4560, [V2, 5.0, MEDIUM])"
234,192.168.1.39,80,open,http,lighttpd,1.4.28,NaN,OK,"(CVE-2014-2324, [V2, 5.0, MEDIUM])"


## 3. Independent python script
<a id='description'></a>
### 3.1. Description
Script is based on code snippets presented above. In addition it implements *click* library functionality. The script imlements following options:<br>
![](https://i.imgur.com/fqDpUcN.png)

- default scan:
![](https://i.imgur.com/eGuqnFf.png)
- *sort* = *False*:
![](https://i.imgur.com/U7KCWZH.png)
- *explode* = *False*:
![](https://i.imgur.com/1so2rBw.png)

<a id='code'></a>
### 3.2. Script code

```python
import nmap
import pandas as pd
import nvdlib
from io import StringIO as sio
import numpy as np
import click

@click.command()
@click.option('--input', '-i', prompt = 'List of hosts', help='Input file which contains list of hosts to scan. If containing several hosts or address spaces(ip/mask), each of them should be seperated with [space]. Default: lista_hostow.')
@click.option('--output', '-o', prompt='Output file', help='Output file to write processed CSV data.')
@click.option('--sort', '-s', default = True, help = 'If you don\'t want to sort found CVEs descending by CVSS then set this parameter to False. Default: True.')
@click.option('--explode', '-e', default = True, help = 'If you want to keep CVEs for each service in a list then set this parameter to False. Default: True.')
def scan(input, output, sort, explode):
    f = open(input)
    nm = nmap.PortScanner()
    print('Scanning... (this may take a while)')
    nm.scan(f.read())
    print('nmap parameters: ' + nm.command_line())
    print('Processing data and searching for CVEs... (this may take a while)')
    data = sio(nm.csv())
    df = pd.read_csv(data, sep=';')
    df = df.loc[:, ['host', 'hostname', 'port', 'state', 'name', 'product', 'version', 'extrainfo', 'cpe']]
    df['status'] = df['cpe'].apply(lambda x: 'Service not found' if x is np.nan else 'Version not found' if x.count(':') == 3 else 'OK' if x.count(':') == 4 else 'Unknown error')
    df['vunerabilities'] = df['cpe'].apply(lambda x: np.nan if x is np.nan else np.nan if x.count(':') == 3 else [(cve.id, cve.score) for cve in nvdlib.searchCVE(cpeName = x)])
    if sort is True:
        df['vunerabilities'] = df['vunerabilities'].apply(lambda x: np.nan if x is np.nan else sorted(x, key = lambda y: y[1][1], reverse = True))
    if explode is True:
        df_exploded = df.explode('vunerabilities', ignore_index = False)
        df_exploded.reset_index(inplace=True)
    else:
        df_exploded = df
    print('Result:')
    print(df_exploded)
    df_exploded.to_csv(output)
    print('Result saved to: ' + output)

if __name__ == '__main__':
    scan()
```